Go beyond manual keywords to uncover real help-seeking language patterns from student discourse.

In [15]:
from pathlib import Path
import sys
import pandas as pd
from IPython.display import display, HTML
from IPython.display import display, HTML
import re


# Set project root to one level above current notebook directory
project_root = Path().resolve().parent
sys.path.append(str(project_root))

from utils.db_connection import get_db_connection

    

In [16]:
# notebook_setup.py

# Imports


# Load data
db = get_db_connection()
df = pd.read_sql_query(
    """
    SELECT p.post_id, p.subreddit_id, p.title, p.selftext, p.created_utc,
           p.score, p.num_comments, p.permalink, s.name AS subreddit_name
    FROM posts p
    LEFT JOIN subreddits s ON p.subreddit_id = s.subreddit_id
    """, db)
db.close()

print(f"Loaded {len(df)} posts from {df['subreddit_name'].nunique()} subreddits")

# Combine and clean text
def combine_and_clean_text(df):
    post_text = (df['title'].fillna('') + ' ' + df['selftext'].fillna('')).str.strip()
    return post_text.str.lower()

df_clean = df.copy()
df_clean['post_text'] = combine_and_clean_text(df_clean)

# Filter for short posts <60 words for manual labeling
df_short = df_clean[df_clean['post_text'].str.split().str.len() < 60]

# NLP setup paths (adjust to your project layout)
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = Path("/Users/buddy/Desktop/WGU-Reddit")
OPENAI_MODULE_PATH = PROJECT_ROOT / "OpenAI"
CONFIG_PATH = OPENAI_MODULE_PATH / "api_config.yaml"

if str(OPENAI_MODULE_PATH) not in sys.path:
    sys.path.insert(0, str(OPENAI_MODULE_PATH))

Loaded 18907 posts from 51 subreddits


In [17]:
def combine_and_clean_text(df):
    """
    Returns a cleaned text Series combining 'title' and 'selftext'.
    """
    post_text = (df['title'].fillna('') + ' ' + df['selftext'].fillna('')).str.strip()
    post_text = post_text.str.lower()
    return post_text

In [18]:
# filename: step1_clean_columns.py

# Keep only necessary columns
df = df[['post_id', 'title', 'selftext']].copy()

# Combine and clean text
def combine_and_clean_text(df):
    return (df['title'].fillna('') + ' ' + df['selftext'].fillna('')).str.lower().str.strip()

df['post_text'] = combine_and_clean_text(df)

# Add columns for manual label and keyword test
df['help_truth'] = 0
df['help_flag'] = 0

print("df columns:", df.columns.tolist())

df columns: ['post_id', 'title', 'selftext', 'post_text', 'help_truth', 'help_flag']


In [19]:
csv_path = Path("/Users/buddy/Desktop/WGU-Reddit/outputs/manual_help_truth.csv")


In [20]:
# filename: step2_sample_for_labeling.py

# Filter for short posts
df['word_count'] = df['post_text'].str.split().str.len()
sample_df = df[df['word_count'] < 50].sample(n=100, random_state=42)

from pathlib import Path

# Define absolute path manually
csv_path = Path("/Users/buddy/Desktop/WGU-Reddit/outputs/manual_help_truth.csv")

# Export sample
sample_df[['post_id', 'post_text']].assign(help_truth=0).to_csv(csv_path, index=False)

print(f"Exported to: {csv_path}")

Exported to: /Users/buddy/Desktop/WGU-Reddit/outputs/manual_help_truth.csv
